# Week #2
## Метрические методы
### Выбор числа соседей
Метрические методы основаны на гипотезе компактности, суть которой состоит в том, что объекты с похожими признаковыми описаниями имеют похожие значения целевой переменной. Если эта гипотеза верна, то строить прогноз для нового объекта можно на основе близких к нему объектов из обучающей выборки — например, путем усреднения их ответов (для регрессии) или путем выбора наиболее популярного среди них класса (для классификации). Методы такого типа и называются метрическими. Они имеют несколько особенностей:

- Процедура обучения, по сути, отсутствует — достаточно лишь запомнить все объекты обучающей выборки
- Можно использовать метрику, учитывающую особенности конкретного набора данных — например, наличие категориальных (номинальных) признаков
- При правильном выборе метрики и достаточном размере обучающей выборки метрические алгоритмы показывают качество, близкое к оптимальному

Метрические методы чувствительны к масштабу признаков — так, если масштаб одного из признаков существенно превосходит масштабы остальных признаков, то их значения практически не будут влиять на ответы алгоритма. Поэтому важно производить масштабирование признаков. Обычно это делается путем вычитания среднего значения признака и деления на стандартное отклонение.

Метод k ближайших соседей реализован в классе sklearn.neighbors.KNeighborsClassifier. Основным параметром является n_neighbors, который задает число соседей для построения прогноза.

Вам понадобится производить кросс-валидацию по блокам. Кросс-валидация заключается в разделении выборки на m непересекающихся блоков примерно одинакового размера, после чего выполняется m шагов. На i-м шаге i-й блок выступает в качестве тестовой выборки, объединение всех остальных блоков — в качестве обучающей выборки. Соответственно, на каждом шаге алгоритм обучается на некоторой обучающей выборке, после чего вычисляется его качество на тестовой выборке. После выполнения m шагов мы получаем m показателей качества, усреднение которых и дает оценку кросс-валидации. Подробнее вы можете послушать про кросс-валидацию в видео "Проблема переобучения. Методология решения задач машинного обучения" из первого модуля, а также почитать на Википедии (на русском или на английском) или в документации scikit-learn.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import scale
from sklearn import datasets

In [3]:
# Загрузите выборку Wine по адресу
# https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data
data = pd.read_csv('data/wine.csv', header=None)
data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185


In [4]:
# Извлеките из данных признаки и классы. Класс записан в первом столбце (три варианта), 
# признаки — в столбцах со второго по последний. Более подробно о сути признаков можно 
# прочитать по адресу https://archive.ics.uci.edu/ml/datasets/Wine (см. также файл wine.names,
# приложенный к заданию)
X = data.ix[:,1:]
y = data[0]

In [5]:
# Оценку качества необходимо провести методом кросс-валидации по 5 блокам (5-fold). 
# Создайте генератор разбиений, который перемешивает выборку перед формированием блоков 
# (shuffle=True). Для воспроизводимости результата, создавайте генератор KFold с фиксированным 
# параметром random_state=42. В качестве меры качества используйте долю верных ответов 
# (accuracy).
# Создается генератор разбиений sklearn.model_selection.KFold, который задает набор разбиений 
# на обучение и валидацию. Число блоков в кросс-валидации определяется параметром n_folds. 
# Обратите внимание, что порядок следования объектов в выборке может быть неслучайным, 
# это может привести к смещенности кросс-валидационной оценки. Чтобы устранить такой эффект, 
# объекты выборки случайно перемешивают перед разбиением на блоки. Для перемешивания 
# достаточно передать генератору KFold параметр shuffle=True.
kf = KFold(shuffle=True, n_splits=5, random_state=42)

In [6]:
# Вычислить качество на всех разбиениях можно при помощи функции 
# sklearn.model_selection.cross_val_score. В качестве параметра estimator передается 
# классификатор, в качестве параметра cv — генератор разбиений с предыдущего шага. 
# С помощью параметра scoring можно задавать меру качества, по умолчанию в задачах 
# классификации используется доля верных ответов (accuracy). Результатом является массив, 
# значения которого нужно усреднить.
# accs = np.empty([0,2], dtype=[('k', float), ('accuracy', float)])
k = -1
max_acc = 0.
for i in range(1,51):
    method = KNeighborsClassifier(n_neighbors=i)
    acc = np.average(cross_val_score(estimator=method, X=X, y=y, cv=kf))
    if max_acc < acc:
        max_acc = acc
        k = i

print 'k={} acc={}'.format(k, max_acc)

k=1 acc=0.730476190476


In [7]:
# Произведите масштабирование признаков с помощью функции sklearn.preprocessing.scale. 
# Снова найдите оптимальное k на кросс-валидации.
# Приведение признаков к одному масштабу можно делать с помощью функции 
# sklearn.preprocessing.scale, которой на вход необходимо подать матрицу признаков 
# и получить масштабированную матрицу, в которой каждый столбец имеет нулевое среднее значение 
# единичное стандартное отклонение.
X = scale(data.ix[:,1:])

In [8]:
# Какое значение k получилось оптимальным после приведения признаков к одному масштабу? 
# Приведите ответы на вопросы 3 и 4. Помогло ли масштабирование признаков?
k = -1
max_acc = 0.
for i in range(1,51):
    method = KNeighborsClassifier(n_neighbors=i)
    acc = np.average(cross_val_score(estimator=method, X=X, y=y, cv=kf))
    if max_acc < acc:
        max_acc = acc
        k = i

print 'k={} acc={}'.format(k, max_acc)

k=29 acc=0.977619047619


### Выбор метрики

Главным параметром любого метрического алгоритма является функция расстояния (или метрика), используемая для измерения сходства между объектами. Можно использовать стандартный вариант (например, евклидову метрику), но гораздо более эффективным вариантом является подбор метрики под конкретную задачу. Один из подходов — использование той же евклидовой метрики, но с весами: каждой координате ставится в соответствие определенный коэффициент; чем он больше, тем выше вклад признака в итоговое расстояние. Веса настраиваются с целью оптимизации качества на отложенной выборке. Другой подход, о котором и пойдет речь в данном задании — выбор метрики из некоторого класса метрик. Мы возьмем за основу метрику Минковского:
dist(x,z)=(sum(xj-zi)^p)^1/p
Параметром метрики Минковского является число p, которое мы и будем настраивать.

Нам понадобится решать задачу регрессии с помощью метода k ближайших соседей — воспользуйтесь для этого классом sklearn.neighbors.KNeighborsRegressor. Метрика задается с помощью параметра metric, нас будет интересовать значение ’minkowski’. Параметр метрики Минковского задается с помощью параметра p данного класса.

Мы будем использовать в данном задании набор данных Boston, где нужно предсказать стоимость жилья на основе различных характеристик расположения (загрязненность воздуха, близость к дорогам и т.д.). Подробнее о признаках можно почитать по адресу https://archive.ics.uci.edu/ml/datasets/Housing


In [9]:
# Загрузите выборку Boston с помощью функции sklearn.datasets.load_boston(). 
# Результатом вызова данной функции является объект, у которого признаки записаны в поле data, 
# а целевой вектор — в поле target.
ds = datasets.load_boston()
X = ds.data
y = ds.target

In [10]:
# Приведите признаки в выборке к одному масштабу при помощи функции 
# sklearn.preprocessing.scale.
X = scale(X)

In [11]:
# Переберите разные варианты параметра метрики p по сетке от 1 до 10 с таким шагом, 
# чтобы всего было протестировано 200 вариантов (используйте функцию numpy.linspace). 
# Качество оценивайте, как и в предыдущем задании, с помощью кросс-валидации по 5 блокам 
# с random_state = 42, не забудьте включить перемешивание выборки (shuffle=True).
kf = KFold(shuffle=True, n_splits=5, random_state=42)
ps = np.linspace(1, 10, num=200)

In [12]:
# Используйте KNeighborsRegressor с n_neighbors=5 и weights='distance' — данный параметр 
# добавляет в алгоритм веса, зависящие от расстояния до ближайших соседей. 
# В качестве метрики качества используйте среднеквадратичную ошибку 
# (параметр scoring='mean_squared_error' у cross_val_score; при использовании библиотеки 
# scikit-learn версии 0.18.1 и выше необходимо указывать scoring='neg_mean_squared_error'). 
# Определите, при каком p качество на кросс-валидации оказалось оптимальным. 
# Обратите внимание, что cross_val_score возвращает массив показателей качества по блокам; 
# необходимо максимизировать среднее этих показателей. 
# Это значение параметра и будет ответом на задачу.
max_p = -1
max_acc = -float('inf')
for p in ps:
    method = KNeighborsRegressor(n_neighbors=5, weights='distance', metric='minkowski', p=p)
    acc = np.average(cross_val_score(estimator=method, X=X, y=y, cv=kf, 
                                     scoring='neg_mean_squared_error'))
    if max_acc < acc:
        max_acc = acc
        max_p = p

print 'p={} acc={}'.format(max_p, max_acc)

p=1.0 acc=-16.0502085084


## Линейные методы классификации
### Нормализация признаков
Линейные алгоритмы — распространенный класс моделей, которые отличается своей простотой и скоростью работы. Их можно обучать за разумное время на очень больших объемах данных, и при этом они могут работать с любыми типами признаков — вещественными, категориальными, разреженными. В этом задании мы предлагаем вам воспользоваться персептроном — одним из простейших вариантов линейных моделей.

Как и в случае с метрическими методами, качество линейных алгоритмов зависит от некоторых свойств данных. В частности, признаки должны быть нормализованы, то есть иметь одинаковый масштаб. Если это не так, и масштаб одного признака сильно превосходит масштаб других, то качество может резко упасть.

Один из способов нормализации заключается в стандартизации признаков. Для этого берется набор значений признака на всех объектах, вычисляется их среднее значение и стандартное отклонение. После этого из всех значений признака вычитается среднее, и затем полученная разность делится на стандартное отклонение.

В библиотеке scikit-learn линейные методы реализованы в пакете sklearn.linear_model. Мы будем работать с реализацией персептрона sklearn.linear_model.Perceptron. Как и у большинства моделей, обучение производится с помощью функции fit, построение прогнозов — с помощью функции predict.

Пример использования:

import numpy as np

from sklearn.linear_model import Perceptron

X = np.array([[1, 2], [3, 4], [5, 6]])

y = np.array([0, 1, 0])

clf = Perceptron()

clf.fit(X, y)

predictions = clf.predict(X)

В качестве метрики качества мы будем использовать долю верных ответов (accuracy). Для ее подсчета можно воспользоваться функцией sklearn.metrics.accuracy_score, первым аргументом которой является вектор правильных ответов, а вторым — вектор ответов алгоритма.

Для стандартизации признаков удобно воспользоваться классом sklearn.preprocessing.StandardScaler. Функция fit_transform данного класса находит параметры нормализации (средние и дисперсии каждого признака) по выборке, и сразу же делает нормализацию выборки с использованием этих параметров. Функция transform делает нормализацию на основе уже найденных параметров.

Пример использования:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = np.array([[100.0, 2.0], [50.0, 4.0], [70.0, 6.0]])

X_test = np.array([[90.0, 1], [40.0, 3], [60.0, 4]])

X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.transform(X_test)

In [13]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [14]:
# Загрузите обучающую и тестовую выборки из файлов perceptron-train.csv и perceptron-test.csv. 
# Целевая переменная записана в первом столбце, признаки — во втором и третьем.
data = pd.read_csv('data/perceptron-train.csv', header=None)
X_train = data.ix[:,1:]
y_train = data.ix[:,0]
data = pd.read_csv('data/perceptron-test.csv', header=None)
X_test = data.ix[:,1:]
y_test = data.ix[:,0]

In [15]:
# Обучите персептрон со стандартными параметрами и random_state=241.
method = Perceptron(random_state=241)
method.fit(X_train, y_train)

Perceptron(alpha=0.0001, class_weight=None, eta0=1.0, fit_intercept=True,
      n_iter=5, n_jobs=1, penalty=None, random_state=241, shuffle=True,
      verbose=0, warm_start=False)

In [16]:
# Подсчитайте качество (долю правильно классифицированных объектов, accuracy) 
# полученного классификатора на тестовой выборке.
predictions = method.predict(X_test)
acc = accuracy_score(y_test, predictions)
print 'Accuracy w/o scaling={}'.format(acc)

Accuracy w/o scaling=0.655


In [17]:
# Нормализуйте обучающую и тестовую выборку с помощью класса StandardScaler.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [18]:
# Обучите персептрон на новой выборке. Найдите долю правильных ответов на тестовой выборке.
# method = Perceptron(random_state=241)
method.fit(X_train_scaled, y_train)
predictions_scaled = method.predict(X_test_scaled)
acc_scaled = accuracy_score(y_test, predictions_scaled)
print 'Accuracy w/ scaling={}'.format(acc_scaled)

Accuracy w/ scaling=0.845


In [19]:
# Найдите разность между качеством на тестовой выборке после нормализации и качеством до нее. 
# Это число и будет ответом на задание.
print 'Accuracy diff={}'.format(acc_scaled-acc)

Accuracy diff=0.19
